## Семинар: "Методы оптимизации"

In [ ]:
from functools import partial
import numpy as np
import torch
from tqdm.auto import tqdm
from torch import nn
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.pyplot import cm
from matplotlib.colors import LogNorm
from matplotlib import animation
from mpl_toolkits import mplot3d
from matplotlib.gridspec import GridSpec
from torchvision import datasets, transforms
from IPython.display import HTML
%matplotlib inline
plt.rc('animation', html='jshtml')

На этом семинаре мы попробуем реализовать и сравнить различные методы оптимизации: SGD, Momentum, NAG, Adagrad, RMSProp, AdaDelta.

### Реализация методов

Для всех экспериментов подберите параметры так, чтобы метод сошелся к ближайшему локальному минимуму. Все методы следует запускать из одной и той же точки.

In [ ]:
# https://github.com/dzlab/deepprojects/blob/master/visualization/Optimizers_in_Action.ipynb

def grid_samples(center=[0, 0], offset=5, size=100):
    range1 = np.linspace(center[0]-offset, center[0]+offset, size)
    range2 = np.linspace(center[1]-offset, center[1]+offset, size)
    return torch.from_numpy(np.stack(np.meshgrid(range1, range2))).float()


def mse(y, y_hat):
    return ((y - y_hat) ** 2).mean(axis=-1)


def msre(y, y_hat):
    return ((y - y_hat) ** 2).mean(axis=-1).sqrt()


def mae(y, y_hat):
    return ((y - y_hat).abs()).mean(axis=-1)


class LossAnimator:
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.size = len(X)

    def loss_func(self, W, loss):
        shape = W.shape
        return loss((self.X @ W.view(shape[0], -1)).T, self.y).view(shape[1:])

    def plot_loss_funcs(self, weights, fcts, titles, view=(20, 50)):
        num_fcts = len(fcts)
        fig = plt.figure(figsize=(7 * num_fcts,7))
        for i in range(num_fcts):
            loss = self.loss_func(weights, loss=fcts[i])
            ax = fig.add_subplot(1, num_fcts, i+1, projection='3d')
            ax.plot_surface(*weights, loss, cmap='viridis')
            ax.set_xlabel('w0'); ax.set_ylabel('w1'); ax.set_zlabel('Loss')
            ax.set_title(titles[i])
            ax.view_init(*view)

    def _init_animation(self, epochs, train_data):
        self.train_data = train_data
        self.epochs = epochs
        self.nmethods = len(train_data)

        weights = grid_samples(offset=5)

        max_loss = max([data['losses'].max() for data in train_data.values()])
        loss_curve = self.loss_func(weights, loss=mse)
        colors = cm.rainbow(np.linspace(0, 1, self.nmethods))


        self.fig = plt.figure(figsize=(14, 8))
        self.gs = GridSpec(2, 2, width_ratios=[1, 2.5])

        self.ax0 = self.fig.add_subplot(self.gs[0,0])
        self.lines0 = {
            name: self.ax0.plot([], [], c=c, label=name)[0]
            for name, c in zip(train_data.keys(), colors)
        }
        self.ax0.scatter(self.X[:,0], self.y, c='orange', label='Ground truth')
        self.ax0.set_ylim(self.y.min(), self.y.max())
        self.ax0.set_title('Ground truth & Model', fontsize=16)
        self.ax0.legend(loc='lower right')

        self.ax1 = self.fig.add_subplot(self.gs[:,1], projection='3d')
        self.ax1.plot_surface(*weights, loss_curve-0.5, cmap='viridis', alpha=0.8)
        self.ax1.view_init(50, 70)
        self.lines1 = {
            name: self.ax1.plot3D([], [], [], c=c, marker='o', alpha=0.9, label=name)[0]
            for name, c in zip(train_data.keys(), colors)
        }
        self.ax1.set_title('Loss', fontsize=16, pad=20)
        self.ax1.set_xlabel('w0')
        self.ax1.set_ylabel('w1')
        self.ax1.set_zlabel('Loss')
        self.ax1.legend()

        self.ax2 = self.fig.add_subplot(self.gs[1,0])
        self.lines2 = {
            name: self.ax2.plot([], [], c=c, label=name)[0]
            for name, c in zip(train_data.keys(), colors)
        }
        self.ax2.set_title('Loss', fontsize=16)
        self.ax2.set_ylabel('loss')
        self.ax2.set_ybound(0, max_loss)
        self.ax2.set_xlim(0, epochs)
        self.ax2.legend(loc='center right')

        self.fig.tight_layout()
        self.fig.subplots_adjust(top=0.85)

        plt.close()

        return self.fig

    def _animate(self, i):
        steps = np.arange(i+1)
        left = max(0, i-20)
        for name, data in self.train_data.items():
            # plot ground truth & model
            self.lines0[name].set_data(self.X[:, 0], self.X @ data['weights'][i])

            # plot loss (output of the sampling)
            self.lines1[name].set_data(data['weights'][left:i+1, 0], data['weights'][left:i+1, 1])
            self.lines1[name].set_3d_properties(data['losses'][left:i+1])

            self.lines2[name].set_data(steps, data['losses'][:i+1])

        self.fig.suptitle(f'Epoch: {i}/{self.epochs}', fontsize=22)

    def animate(self, epochs, train_data, step_skip=1):
        self._init_animation(epochs, train_data)
        anim = animation.FuncAnimation(
            self.fig, self._animate,
            frames=range(0, epochs, step_skip),
            interval=100 * step_skip
        )
        return HTML(anim.to_jshtml())

In [ ]:
size = 100
X = torch.ones(size, 2)
X[:, 0].uniform_(-1., 1)

y_hat = 3 * X[:, 0] + 2
y = y_hat + torch.randn(size)

loss_animator = LossAnimator(X, y)

plt.scatter(loss_animator.X[:,0], loss_animator.y, label='y');
plt.scatter(loss_animator.X[:,0], y_hat, label='y_hat');
plt.xlabel('X')
plt.ylabel('y')
plt.legend()

In [ ]:
weights = grid_samples()
loss_animator.plot_loss_funcs(weights, [mse, msre, mae], ['MSE', 'MSRE', 'MAE'])

In [ ]:
class Optimizer:
    def __init__(self, params):
        self.params = list(params)

    def zero_grad(self):
        for param in self.params:
            if param.grad is not None:
                param.grad.detach_()
                param.grad.zero_()

    def pre_step(self):
        pass

    def step(self):
        pass

    def update_param(self, p):
        raise NotImplementedError()


In [ ]:
def optimize_function(fn, optim, optim_args, start_point, num_iter = 50):
    weigths = nn.Parameter(torch.FloatTensor(start_point), requires_grad=True)

    optim = optim(params=[weigths], **optim_args)
    points = []
    losses = []
    for i in range(num_iter):
        if hasattr(optim, 'pre_step'):
            optim.pre_step()
        loss = fn(weigths)
        points.append(weigths.data.detach().clone())
        losses.append(loss.item())
        loss.backward()
        optim.step()
        optim.zero_grad()
    points = torch.stack(points, axis=0)
    losses = torch.FloatTensor(losses)
    return points, losses


In [ ]:
def compare_optimizers(
    loss_animator,
    fn,
    optim_list,
    start_point,
    num_iter = 50,
    step_skip = 1
):
    data = {}
    loss_func = partial(loss_animator.loss_func, loss=fn)
    for name, optim, args in tqdm(optim_list):
        points, losses = optimize_function(loss_func, optim, args, start_point, num_iter)
        data[name] = {
            'weights': points,
            'losses': losses,
        }

    return loss_animator.animate(num_iter, data, step_skip=step_skip)

#### SGD
$$\theta_t = \theta_{t-1} - \eta \sum_{i_1, ..., i_k} \nabla_{\theta} J_{i} (\theta_{t-1})$$

In [ ]:
class SGD(Optimizer):
    def __init__(self, params, lr=1e-2):
        super().__init__(params)
        self.lr = lr

    def step(self):
        with torch.no_grad():
            for p in self.params:
                if p.grad is None:
                    continue
                p -= self.lr * p.grad


#### Momentum
$$\nu_t=\gamma \nu_{t-1} + \eta_t \nabla_{\theta} J_{i} (\theta_{t-1})$$
$$\theta_t = \theta_{t-1} - \nu_t$$

In [ ]:
class Momentum(Optimizer):
    def __init__(self, params, lr=1e-2, gamma=0.9):
        super().__init__(params)
        self.lr = lr
        self.gamma = gamma

        self.prev_momentum = [torch.zeros(p.shape) for p in self.params]

    def step(self):
        with torch.no_grad():
            for i, p in enumerate(self.params):
                if p.grad is None:
                    continue

                prev_momentum = self.prev_momentum[i]
                self.prev_momentum[i] = self.gamma * prev_momentum + self.lr * p.grad
                p -= self.prev_momentum[i]

#### Nesterov Accelerated Gradient (NAG)
$$\nu_t=\gamma \nu_{t-1} + \eta_t \nabla_{\theta} J_{i} (\theta_{t-1} - \gamma \nu_{t-1})$$
$$\theta_t = \theta_{t-1} - \nu_t$$

In [ ]:
class NAG(Optimizer):
    def __init__(self, params, lr=1e-2, gamma=0.9):
        super().__init__(params)
        self.lr = lr
        self.gamma = gamma

        self.prev_momentum = [torch.zeros(p.shape) for p in self.params]

    def step(self):
        with torch.no_grad():
            for i, p in enumerate(self.params):
                if p.grad is None:
                    continue

                prev_momentum = self.prev_momentum[i]
                self.prev_momentum[i] = self.gamma * prev_momentum + self.lr * p.grad
                p -= (1 + self.gamma) * self.prev_momentum[i] - self.gamma * prev_momentum

#### AdaGrad
$$G_t = \sum_{k=0}^t g_{k}^2$$
$$\theta_{t} = \theta_{t-1} - \frac{\eta}{\sqrt{G_{t-1} + ϵ}} \cdot g_{t-1}$$

In [ ]:
class AdaGrad(Optimizer):
    def __init__(self, params, epsilon=1e-8, eta=1e-2):
        super().__init__(params)
        self.epsilon = epsilon
        self.eta = eta

        self.G = [torch.zeros(p.shape) for p in self.params]

    def step(self):
        with torch.no_grad():
            for i, p in enumerate(self.params):
                if p.grad is None:
                    continue
                self.G[i] += p.grad ** 2
                p -= self.eta * p.grad / torch.sqrt(self.G[i] + self.epsilon)

#### RMSProp
$$G_t = \gamma G_{t-1} + (1 - \gamma) g_t^2$$
$$\theta_{t} = \theta_{t-1} - \frac{\eta}{\sqrt{G_{t-1} + ϵ}} \cdot g_{t-1}$$

In [ ]:
class RMSProp(Optimizer):
    def __init__(self, params, epsilon=1e-8, eta=1e-2, gamma=0.9):
        super().__init__(params)
        self.eta = eta
        self.epsilon = epsilon
        self.gamma = gamma

        self.G = [torch.zeros(p.shape) for p in self.params]


    def step(self):
        with torch.no_grad():
            for i, p in enumerate(self.params):
                if p.grad is None:
                    continue

                self.G[i] = self.gamma * self.G[i] + (1 - self.gamma) * (p.grad ** 2)
                p -= self.eta / torch.sqrt(self.G[i] + self.epsilon) * p.grad

In [ ]:
from functools import partial

points, losses = optimize_function(
    partial(loss_animator.loss_func, loss=mse),
    SGD,
     {'lr': 1e-2},
    start_point=[-4, -4],
    num_iter=100
)

In [ ]:
compare_optimizers(
    loss_animator,
    mae,
    [
        ['SGD', SGD, {'lr': 1e-1}],
        ['Momentum', NAG, {'lr': 1e-1}],
        ['NAG', NAG, {'lr': 1e-1}],
        ['AdaGrad', AdaGrad, {'eta' : 1}],
        ['RMSProp', RMSProp, {'eta' : 1}],
    ],
    start_point=[-4, -4],
    num_iter=100
)

###  Домашнее задание: Обучение нейронной сети + Подбор шага

В этом задании вам нужно:
1) Реализовать Adam, AdamW и обучить сверточную нейросеть на MNIST. Сравнить графики обучения для SGD, Adam и AdamW. **(2 балла)**
2) Реализовать LinearLR и CosineAnnealingLR. Обучить сверточную нейросеть на MNIST с SGD, Adam, AdamW с LinearLR, CosineAnnealingLR. Сравните графики изменения loss-функции и точности (суммарно должно быть 9 экспериментов, 3 из которых покрываются пунктом 1). **(3 балла)**

Ссылки:
- https://arxiv.org/abs/1711.05101 (AdamW, статья)
- https://docs.pytorch.org/docs/stable/generated/torch.optim.AdamW.html (AdamW, PyTorch)
- https://docs.pytorch.org/docs/stable/generated/torch.optim.lr_scheduler.LinearLR.html (LinearLR, PyTorch)
- https://docs.pytorch.org/docs/stable/generated/torch.optim.lr_scheduler.CosineAnnealingLR.html (CosineAnnealingLR, PyTorch)

In [ ]:
!wget www.di.ens.fr/~lelarge/MNIST.tar.gz
!tar -zxvf MNIST.tar.gz

In [ ]:
transform = transforms.Compose([
   transforms.ToTensor(),
   transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST('.', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('.', train=False, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True)

In [ ]:
class Adam(Optimizer):
    def __init__(self, params, lr = 1e-2, beta1 = 0.9, beta2 = 0.999, eta = 1e-8):
        super().__init__(params)
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.eta = eta
        self.prev_momentum = [torch.zeros(p.shape) for p in self.params]
        self.G = [torch.zeros(p.shape) for p in self.params]
        self.t = 0

    def step(self):
        self.t += 1
        with torch.no_grad():
            for i, p in enumerate(self.params):
                if p.grad is None:
                    continue
                prev_momentum = self.prev_momentum[i]
                self.prev_momentum[i] = self.beta1 * prev_momentum + (1 - self.beta1) * p.grad
                self.G[i] = self.beta2 * self.G[i] + (1 - self.beta2) * (p.grad ** 2)
                m_n = self.prev_momentum[i] / (1 - self.beta1 ** self.t)
                v_n = self.G[i] / (1 - self.beta2 ** self.t)
                p -= self.lr * m_n / (v_n ** 0.5 + self.eta)

In [ ]:
class AdamW(Optimizer):
    def __init__(self, params, lr = 1e-2, wd = 1e-1, beta1 = 0.9, beta2 = 0.999, eta = 1e-8):
        super().__init__(params)
        self.lr = lr
        self.wd = wd
        self.beta1 = beta1
        self.beta2 = beta2
        self.eta = eta
        self.prev_momentum = [torch.zeros(p.shape) for p in self.params]
        self.G = [torch.zeros(p.shape) for p in self.params]
        self.t = 0

    def step(self):
        self.t += 1
        with torch.no_grad():
            for i, p in enumerate(self.params):
                if p.grad is None:
                    continue
                prev_momentum = self.prev_momentum[i]
                self.prev_momentum[i] = self.beta1 * prev_momentum + (1 - self.beta1) * p.grad
                self.G[i] = self.beta2 * self.G[i] + (1 - self.beta2) * (p.grad ** 2)
                m_n = self.prev_momentum[i] / (1 - self.beta1 ** self.t)
                v_n = self.G[i] / (1 - self.beta2 ** self.t)
                p -= self.lr * m_n / (v_n ** 0.5 + self.eta)
                p -= self.lr * self.wd * p

In [ ]:
compare_optimizers(
    loss_animator,
    mae,
    [
        ['SGD', SGD, {'lr': 1e-1}],
        ['Adam', Adam, {'lr': 1e-1}],
        ['AdamW', AdamW, {'lr': 1e-1}],
    ],
    start_point=[-4, -4],
    num_iter=100
)

In [ ]:
class LRScheduler:
    def __init__(self, optimizer, mode="const", total_iters=100, start_factor=0.1, end_factor=1.0, eta_min=1e-5):
        pass #Пытался через Scheduler, но не смог до конца довести

    def step(self):
        pass

In [ ]:
class LinearLR(LRScheduler):
    def __init__(
        self,
        optimizer: Optimizer,
        start_factor: float = 0.1,
        end_factor: float = 1.0,
        total_iters: int = 100
    ):
        super().__init__(optimizer)
        self.start_factor = start_factor
        self.end_factor = end_factor
        self.total_iters = total_iters
        
    def get_factor(self, t: int) -> float:
        if self.total_iters <= 0:
            return self.end_factor
        t_eff = min(max(t, 0), self.total_iters)
        return self.start_factor + (t_eff / self.total_iters) * (self.end_factor - self.start_factor)
        

class CosineAnnealingLR(LRScheduler):
    def __init__(
        self,
        optimizer: Optimizer,
        T_max: int = 100,
        eta_min: float = 0.0
    ):
        super().__init__(optimizer)
        self.T_max = int(T_max)
        self.eta_min = float(eta_min)

    def get_factor(self, t: int) -> float:
        if self.T_max <= 0:
            return self.eta_min / self.base_lr
        t_eff = min(max(t, 0), self.T_max)
        return (self.eta_min / self.base_lr) + 0.5 * (1 - self.eta_min / self.base_lr) * (1 + math.cos(math.pi * t_eff / self.T_max))

In [ ]:
class SGDLinear(Optimizer):
    def __init__(self, params, base = 5e-3, end = 1e-2, n_steps = 10):
        super().__init__(params)
        self.t = 0
        self.base = base
        self.lr = base
        self.end = end
        self.n_steps = n_steps

    def step(self):
        self.t += 1
        if self.t <= self.n_steps:
            self.lr += (self.end - self.base) / self.n_steps
        with torch.no_grad():
            for p in self.params:
                if p.grad is None:
                    continue
                p -= self.lr * p.grad

In [ ]:
class AdamLinear(Optimizer):
    def __init__(self, params, base = 5e-3, end = 1e-2, n_steps = 10, beta1 = 0.9, beta2 = 0.999, eta = 1e-8):
        super().__init__(params)
        self.base = base
        self.lr = base
        self.end = end
        self.n_steps = n_steps
        self.beta1 = beta1
        self.beta2 = beta2
        self.eta = eta
        self.prev_momentum = [torch.zeros(p.shape) for p in self.params]
        self.G = [torch.zeros(p.shape) for p in self.params]
        self.t = 0

    def step(self):
        self.t += 1
        if self.t <= self.n_steps:
            self.lr += (self.end - self.base) / self.n_steps
        with torch.no_grad():
            for i, p in enumerate(self.params):
                if p.grad is None:
                    continue
                prev_momentum = self.prev_momentum[i]
                self.prev_momentum[i] = self.beta1 * prev_momentum + (1 - self.beta1) * p.grad
                self.G[i] = self.beta2 * self.G[i] + (1 - self.beta2) * (p.grad ** 2)
                m_n = self.prev_momentum[i] / (1 - self.beta1 ** self.t)
                v_n = self.G[i] / (1 - self.beta2 ** self.t)
                p -= self.lr * m_n / (v_n ** 0.5 + self.eta)

In [ ]:
class AdamWLinear(Optimizer):
    def __init__(self, params, base = 5e-3, end = 1e-2, n_steps = 10, wd = 1e-1, beta1 = 0.9, beta2 = 0.999, eta = 1e-8):
        super().__init__(params)
        self.base = base
        self.lr = base
        self.end = end
        self.n_steps = n_steps
        self.wd = wd
        self.beta1 = beta1
        self.beta2 = beta2
        self.eta = eta
        self.prev_momentum = [torch.zeros(p.shape) for p in self.params]
        self.G = [torch.zeros(p.shape) for p in self.params]
        self.t = 0

    def step(self):
        self.t += 1
        if self.t <= self.n_steps:
            self.lr += (self.end - self.base) / self.n_steps
        with torch.no_grad():
            for i, p in enumerate(self.params):
                if p.grad is None:
                    continue
                prev_momentum = self.prev_momentum[i]
                self.prev_momentum[i] = self.beta1 * prev_momentum + (1 - self.beta1) * p.grad
                self.G[i] = self.beta2 * self.G[i] + (1 - self.beta2) * (p.grad ** 2)
                m_n = self.prev_momentum[i] / (1 - self.beta1 ** self.t)
                v_n = self.G[i] / (1 - self.beta2 ** self.t)
                p -= self.lr * m_n / (v_n ** 0.5 + self.eta)
                p -= self.lr * self.wd * p

In [ ]:
compare_optimizers(
    loss_animator,
    mae,
    [
        ['SGDLinear', SGDLinear, {'base': 5e-2,  'end': 5e-1}],
        ['AdamLinear', AdamLinear, {'base': 5e-2,  'end': 5e-1}],
        ['AdamWLinear', AdamWLinear, {'base': 5e-2,  'end': 5e-1}],
    ],
    start_point=[-4, -4],
    num_iter=100
)

In [ ]:
class SGDCosine(Optimizer):
    def __init__(self, params, base = 5e-3, end = 1e-2, n_steps = 10):
        super().__init__(params)
        self.t = 0
        self.base = base
        self.lr = base
        self.end = end
        self.n_steps = n_steps

    def step(self):
        self.t += 1
        if self.t <= self.n_steps:
            self.lr = self.base + (self.end - self.base) * (1 + np.cos(self.t * np.pi / self.n_steps)) / 2
        with torch.no_grad():
            for p in self.params:
                if p.grad is None:
                    continue
                p -= self.lr * p.grad

In [ ]:
class AdamCosine(Optimizer):
    def __init__(self, params, base = 5e-3, end = 1e-2, n_steps = 10, beta1 = 0.9, beta2 = 0.999, eta = 1e-8):
        super().__init__(params)
        self.base = base
        self.lr = base
        self.end = end
        self.n_steps = n_steps
        self.beta1 = beta1
        self.beta2 = beta2
        self.eta = eta
        self.prev_momentum = [torch.zeros(p.shape) for p in self.params]
        self.G = [torch.zeros(p.shape) for p in self.params]
        self.t = 0

    def step(self):
        self.t += 1
        if self.t <= self.n_steps:
            self.lr = self.base + (self.end - self.base) * (1 + np.cos(self.t * np.pi / self.n_steps)) / 2
        with torch.no_grad():
            for i, p in enumerate(self.params):
                if p.grad is None:
                    continue
                prev_momentum = self.prev_momentum[i]
                self.prev_momentum[i] = self.beta1 * prev_momentum + (1 - self.beta1) * p.grad
                self.G[i] = self.beta2 * self.G[i] + (1 - self.beta2) * (p.grad ** 2)
                m_n = self.prev_momentum[i] / (1 - self.beta1 ** self.t)
                v_n = self.G[i] / (1 - self.beta2 ** self.t)
                p -= self.lr * m_n / (v_n ** 0.5 + self.eta)

In [ ]:
class AdamWCosine(Optimizer):
    def __init__(self, params, base = 5e-3, end = 1e-2, n_steps = 10, wd = 1e-1, beta1 = 0.9, beta2 = 0.999, eta = 1e-8):
        super().__init__(params)
        self.base = base
        self.lr = base
        self.end = end
        self.n_steps = n_steps
        self.wd = wd
        self.beta1 = beta1
        self.beta2 = beta2
        self.eta = eta
        self.prev_momentum = [torch.zeros(p.shape) for p in self.params]
        self.G = [torch.zeros(p.shape) for p in self.params]
        self.t = 0

    def step(self):
        self.t += 1
        if self.t <= self.n_steps:
            self.lr = self.base + (self.end - self.base) * (1 + np.cos(self.t * np.pi / self.n_steps)) / 2
        with torch.no_grad():
            for i, p in enumerate(self.params):
                if p.grad is None:
                    continue
                prev_momentum = self.prev_momentum[i]
                self.prev_momentum[i] = self.beta1 * prev_momentum + (1 - self.beta1) * p.grad
                self.G[i] = self.beta2 * self.G[i] + (1 - self.beta2) * (p.grad ** 2)
                m_n = self.prev_momentum[i] / (1 - self.beta1 ** self.t)
                v_n = self.G[i] / (1 - self.beta2 ** self.t)
                p -= self.lr * m_n / (v_n ** 0.5 + self.eta)
                p -= self.lr * self.wd * p

In [ ]:
compare_optimizers(
    loss_animator,
    mae,
    [
        ['SGDCosine', SGDCosine, {'base': 5e-2,  'end': 5e-1}],
        ['AdamCosine', AdamCosine, {'base': 5e-2,  'end': 5e-1}],
        ['AdamWCosine', AdamWCosine, {'base': 5e-2,  'end': 5e-1}],
    ],
    start_point=[-4, -4],
    num_iter=100
)

#### Feedback (опционально)

Здесь вы можете оставить список опечаток из лекции или семинара:

- Your text here

Здесь вы можете оставить комментарии по лекции или семинару:

- Your text here